In [55]:
import pandas as pd
import re
from Bio import SeqIO

In [57]:
def fetch_sequence(identifier, fasta_file):
    try:
        # Parse the FASTA file
        for record in SeqIO.parse(fasta_file, "fasta"):
            if record.id == identifier:
                return str(record.seq)
        # If the loop completes, the identifier was not found
        print(f"Identifier '{identifier}' not found in {fasta_file}.")
        return None
    except FileNotFoundError:
        print(f"The file {fasta_file} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [83]:
df = pd.read_csv("acc2operons", sep=" ", header=None) #mapping of uniprot accession to operon_mapper ids, plus file to find operons

operon_hits = {}
no_operon = []
for index, row in df.iterrows():
    acc = row[0]
    gene_id = row[1]
    file = row[2]
    fasta = re.sub("list_of_operons_", "predicted_protein_sequences_", file)#fasta file generated by operon mapper for all cds
    with open(file, 'r') as f:
        lines = f.readlines()
        i = 0
        for line in lines:
            i += 1
            if i > 1: # not the header line
                if str(line)[0].isnumeric(): #start of operon
                    hit = False # default until we know if gene of interest is in operon
                    operon_id = str(line)
                    genes = []
                else: #gene wihtin operon
                    gene = re.sub('\s+',' ',line)#get rid of \t and \n characters
                    gene = gene.strip() #get rid of leading/tailing spaces
                    if gene_id in gene:  hit = True
                    genes.append(gene) #list of genes within operon
                    
                if hit == True and len(genes) < 2: #if no operon was detected for gene of interest, save in list
                    no_operon.append(acc)
                    
                if hit == True and len(genes) >1: #ifoperon was detected for gene of interest, get sequences for analysis
                    
                    out_seqs = []
                    for gene in genes:
                        gid = gene.split()[0].strip()
                        seq = fetch_sequence(gid, fasta)
                        out_seqs.append(seq)
                        
                        
                    operon_hits[acc] = out_seqs
                    print(acc, len(out_seqs))
                        
                        
                        
                            

operon_hits                  
    

F7LW98 2
T4VLI6 2
A0A0D1IL93 2
A0A1F0PND0 2
A0A1F0PND0 3
A0A246E3R1 2
A0A246E3R1 3
A0A246E3R1 4
A0A246E3R1 5
A0A246E3R1 6
A0A246E3R1 7
A0A246E3R1 8
A0A1Q2LGR2 3


{'F7LW98': ['MIKVNFSKPLLIQSVAFKDVFLRMDGNGITKTNGAGAGKVSCQKNMSPTGAFKVQEQKNGTFTIESVKYPGVFLRMDGSGIRGFAGSGAGHVNCQFGASDWEHFKLHEQNDGSYSIESAAFPNVFLRMDGNNRSGKEEDFGTVNCQYGASTCEKFYLLNMPETGKVKDMFNKFAK*',
  'MKVKTFNFEDKALENKALEAHKRLYFYLNRAKLTLERLALLVKSPDNIPNTDLKEVERYKKWFGDFNASNVKEVTRIITEIHKKFEEEKICLYYATKEPGAEPNDFAYVEQDDIDQQILNIYLCNMFFRTKENTERFNTALGTILHELTHLIGKTDDVEIIDCNKKQTTCYGEELCTYLASFNSKLAIKNADNYEMYLETFRNM*'],
 'T4VLI6': ['MNIDSYYRDKEKNNKKVKIKSVYADEFVVVGRDGYLYATGEKGNKKGIFIININDNGKAKIKLEDGPYIRLDNKNFLIADTDKEGATKFDVYKTDSKEYVLKAPNGYYVRVREDDYKLAAKAEGTGEKTKFKFKEVD*',
  'MKYANYKFGNIVDISDIKGQPYKFITIATNPKDIYTFTKLIYSDQTLIVDCEYNEISIDDLEIGDKIVAFHSNVMTMSIPPQTQVYIIEVK*'],
 'A0A0D1IL93': ['MTSLGEMRLALTSPSYNKFDCGENRSVQTYGYGLYEVRMKPAKNTGIVSSFFTYTGPTDGTPWDEIDIEFLGKDTTKVQFNYYTNGAGNHEKIVDLGFDAANAYHTYAFDWQPNSIKWYVDGQLKHTATNQIPTTPGKIMMNLWNGTGIDEWLGSYNGVNPLYAHYDWVRYTKK*',
  'MPYLKRVLLLLVTGLFMSLFAVTSLPRLKQVDRFLTRLTAITPVFGKKQMVIRMEICSTARGGLITYQ*'],
 'A0A1F0PND0': ['LKNKKENCGGKEWSTASIEVRKPFLYGYFECRLKYAQASGVNNSFWFYQSSKANT